# EDA – Planification des rendez-vous médicaux

Ce notebook explore le jeu de données brut : schéma des données, valeurs manquantes, distribution de la variable cible et vérification des fuites de données (data leakage).

In [1]:
import pandas as pd
from pathlib import Path

# Chemin vers le fichier de données brutes (CSV)
raw_path = Path('../data/raw/appointments.csv')

# Chargement du jeu de données dans un DataFrame pandas
df = pd.read_csv(raw_path)

# Affichage des 10 premières lignes du DataFrame
# pour avoir un aperçu de la structure et du contenu des données
df.head(10)

,appointment_id,slot_id,scheduling_date,appointment_date,appointment_time,scheduling_interval,status,check_in_time,appointment_duration,start_time,end_time,waiting_time,patient_id,sex,age,age_group
0,138,1,2014-12-28,2015-01-01,08:00:00,4,did not attend,NaN,NaN,NaN,NaN,NaN,8285,Male,37,35-39
1,146,23,2014-12-29,2015-01-01,13:30:00,3,did not attend,NaN,NaN,NaN,NaN,NaN,5972,Male,84,80-84
2,21,24,2014-12-17,2015-01-01,13:45:00,15,attended,13:36:45,5.2,13:37:57,13:43:09,1.2,6472,Male,77,75-79
3,233,25,2014-12-31,2015-01-01,14:00:00,1,attended,13:59:32,28.9,14:00:40,14:29:34,1.1,5376,Female,37,35-39
4,90,26,2014-12-26,2015-01-01,14:15:00,6,cancelled,NaN,NaN,NaN,NaN,NaN,8028,Male,72,70-74
5,180,27,2014-12-30,2015-01-01,14:30:00,2,attended,14:08:53,7.7,14:30:38,14:38:20,21.7,4317,Female,51,50-54
6,197,28,2014-12-30,2015-01-01,14:45:00,2,attended,14:22:59,4.2,14:39:14,14:43:26,16.2,7638,Male,28,25-29
7,191,29,2014-12-30,2015-01-01,15:00:00,2,attended,14:59:06,27.1,15:00:08,15:27:14,1.0,7061,Male,33,30-34
8,135,30,2014-12-28,2015-01-01,15:15:00,4,cancelled,NaN,NaN,NaN,NaN,NaN,2475,Female,29,25-29
9,130,22,2014-12-28,2015-01-01,13:15:00,4,attended,13:05:52,1.2,13:14:25,13:15:37,8.5,4217,Female,90,90+


In [2]:
# Affiche la dimension du DataFrame :
# (nombre de lignes, nombre de colonnes)
df.shape 

# Affiche la liste des noms des colonnes du DataFrame
df.columns

Index(['appointment_id', 'slot_id', 'scheduling_date', 'appointment_date',
       'appointment_time', 'scheduling_interval', 'status', 'check_in_time',
       'appointment_duration', 'start_time', 'end_time', 'waiting_time',
       'patient_id', 'sex', 'age', 'age_group'],
      dtype='object')

In [3]:
# Calcule la proportion de valeurs manquantes pour chaque colonne
# isna() : détecte les valeurs manquantes
# mean() : calcule le pourcentage de valeurs manquantes par colonne
# sort_values() : trie les colonnes par ordre décroissant
# head(20) : affiche les 20 colonnes avec le plus de valeurs manquantes
df.isna().mean().sort_values(ascending=False).head(20)

check_in_time           0.22833
end_time                0.22833
start_time              0.22833
appointment_duration    0.22833
waiting_time            0.22833
appointment_id          0.00000
scheduling_date         0.00000
slot_id                 0.00000
status                  0.00000
scheduling_interval     0.00000
appointment_time        0.00000
appointment_date        0.00000
patient_id              0.00000
sex                     0.00000
age                     0.00000
age_group               0.00000
dtype: float64

In [4]:
# Génère des statistiques descriptives pour toutes les colonnes du DataFrame
# include='all' : inclut les variables numériques et catégorielles
# transpose() : transpose le tableau pour avoir une ligne par variable
# head(30) : affiche les 30 premières variables
df.describe(include='all').transpose().head(30)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
appointment_id,111488.0,NaN,NaN,NaN,55744.5,32183.957743,1.0,27872.75,55744.5,83616.25,111488.0
slot_id,111488.0,NaN,NaN,NaN,51854.320438,29945.828944,1.0,25909.75,51852.0,77796.25,104159.0
scheduling_date,111488,3642,2024-12-01,166,NaN,NaN,NaN,NaN,NaN,NaN,NaN
appointment_date,111488,2604,2017-10-20,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
appointment_time,111488,40,17:45:00,2811,NaN,NaN,NaN,NaN,NaN,NaN,NaN
scheduling_interval,111488.0,NaN,NaN,NaN,7.190146,6.146793,1.0,2.0,5.0,10.0,30.0
status,111488,5,attended,86032,NaN,NaN,NaN,NaN,NaN,NaN,NaN
check_in_time,86032,33251,15:40:29,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
appointment_duration,86032.0,NaN,NaN,NaN,17.479525,11.064225,0.0,8.6,15.8,24.7,58.7
start_time,86032,35984,08:01:00,123,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Choix de la variable cible

Définir TARGET_COL comme la colonne que l’on souhaite prédire (status).

In [5]:
TARGET_COL = 'status'
if TARGET_COL in df.columns:
    df[TARGET_COL].value_counts(dropna=False).head(20)
else:
    print('Update TARGET_COL. Available columns:', list(df.columns))

In [7]:
# Afficher toutes les valeurs uniques de la colonne 'status'
print(df['status'].unique())

['did not attend' 'attended' 'cancelled' 'unknown' 'scheduled']


In [8]:
# Définir les valeurs à garder
valid_status = {"did not attend", "attended", "cancelled"}

# Filtrer le DataFrame
df_filtered = df[df['status'].isin(valid_status)]

In [9]:
# Sauvegarder le résultat dans un nouveau CSV
df_filtered.to_csv("../data/raw/appointments_filtre.csv", index=False)

print(f"Lignes conservées : {len(df_filtered)}")

Lignes conservées : 110901


In [10]:
raw_path2 = Path('../data/raw/appointments_filtre.csv')
df = pd.read_csv(raw_path2)
df.head(10)

,appointment_id,slot_id,scheduling_date,appointment_date,appointment_time,scheduling_interval,status,check_in_time,appointment_duration,start_time,end_time,waiting_time,patient_id,sex,age,age_group
0,138,1,2014-12-28,2015-01-01,08:00:00,4,did not attend,NaN,NaN,NaN,NaN,NaN,8285,Male,37,35-39
1,146,23,2014-12-29,2015-01-01,13:30:00,3,did not attend,NaN,NaN,NaN,NaN,NaN,5972,Male,84,80-84
2,21,24,2014-12-17,2015-01-01,13:45:00,15,attended,13:36:45,5.2,13:37:57,13:43:09,1.2,6472,Male,77,75-79
3,233,25,2014-12-31,2015-01-01,14:00:00,1,attended,13:59:32,28.9,14:00:40,14:29:34,1.1,5376,Female,37,35-39
4,90,26,2014-12-26,2015-01-01,14:15:00,6,cancelled,NaN,NaN,NaN,NaN,NaN,8028,Male,72,70-74
5,180,27,2014-12-30,2015-01-01,14:30:00,2,attended,14:08:53,7.7,14:30:38,14:38:20,21.7,4317,Female,51,50-54
6,197,28,2014-12-30,2015-01-01,14:45:00,2,attended,14:22:59,4.2,14:39:14,14:43:26,16.2,7638,Male,28,25-29
7,191,29,2014-12-30,2015-01-01,15:00:00,2,attended,14:59:06,27.1,15:00:08,15:27:14,1.0,7061,Male,33,30-34
8,135,30,2014-12-28,2015-01-01,15:15:00,4,cancelled,NaN,NaN,NaN,NaN,NaN,2475,Female,29,25-29
9,130,22,2014-12-28,2015-01-01,13:15:00,4,attended,13:05:52,1.2,13:14:25,13:15:37,8.5,4217,Female,90,90+
